In [1]:
import numpy as np
import numba as nb
import random
import matplotlib.pyplot as plt

In [2]:
p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]

identity_matrix = np.zeros((2,2), dtype = 'complex128')
identity_matrix[0,0]= 1
identity_matrix[1,1] = 1

In [3]:
@nb.jit(nopython=True)
def norm_diff(L, M):
    LminusM = L - M
    mag_LminusM = np.sqrt(np.sum(np.abs(LminusM)**2))
    mag_L = np.sqrt(np.sum(np.abs(L)**2))
    mag_M = np.sqrt(np.sum(np.abs(M)**2))
    
    delta = mag_LminusM / (mag_L + mag_M)
    
    return delta

def matrix_maker(m0xyz): #array to matrix
    M0, Mx, My, Mz = m0xyz
    
    identity_matrix = np.zeros((2,2), dtype = 'complex128')
    identity_matrix[0,0]= 1
    identity_matrix[1,1] = 1

    pauli_x = np.zeros((2,2), dtype = 'complex128')
    pauli_x[0,1]= 1
    pauli_x[1,0] = 1

    pauli_y = np.zeros((2,2), dtype = 'complex128')
    pauli_y[0,1]= -1j
    pauli_y[1,0] = 1j

    pauli_z = np.zeros((2,2), dtype = 'complex128')
    pauli_z[0,0]= 1
    pauli_z[1,1] = -1
    
    m = np.array([Mx, My, Mz])
    
    pauli_basis = np.array([pauli_x, pauli_y, pauli_z])
    
    m_dot_pauli = np.tensordot(m, pauli_basis, axes=1)
    
    matrix = M0 * identity_matrix + m_dot_pauli
    
    return matrix

@nb.jit(nopython=True)
def random_complex_numbers():
    real_parts = np.random.randn(4)
    imaginary_parts = np.random.randn(4)
    complex_numbers = real_parts + 1j*imaginary_parts
    return complex_numbers

In [4]:
######### KEY:
######### takes two, 4 component vectors each corresponding to a matrix
######### multiplies the "matrices" but keeps them as vector/array representations the whole time
######### outputs the components for the resulting matrix
@nb.jit(nopython=True)
def multiply_matrices(M1components, M2components): 
    M10, M1x, M1y, M1z = M1components
    M20, M2x, M2y, M2z = M2components
    m1xyz = np.array([M1x, M1y, M1z])
    m2xyz = np.array([M2x, M2y, M2z])
    m1_dot_m2 = M1x*M2x + M1y*M2y + M1z*M2z
    M30 = M10*M20 + m1_dot_m2
    #M30 = M10*M20 + np.tensordot(m1xyz, m2xyz, axes=1) -- numba didnt like this
    m3xyz = M10*m2xyz + M20*m1xyz +1j*(np.cross(m1xyz, m2xyz))
    M3components = np.array([M30, m3xyz[0], m3xyz[1], m3xyz[2]])
    return M3components

#functions to take p arrays and make general a and b, so that we can do multiplication noramllly

######### KEY:
######### this function generalizes the specific array that is (identity minus rho)...
######### it takes the 4 component vector/array that would create a matrix that is in the form (1-rho)
######### makes it into more generalized components "a0xyz"
@nb.jit(nopython=True)
def generalize_IDrho(IDrho_array):
    P0, Px, Py, Pz = IDrho_array
    pxyz = np.array([Px, Py, Pz])
    A0 = 1 - 0.5*P0
    axyz = -0.5*P0*pxyz
    a0xyz = np.array([A0, axyz[0], axyz[1], axyz[2]])
    return a0xyz

######### KEY:
######### this function generalizes the specific array that is just rho...
######### it takes the 4 component vector/array that would create a matrix that is in the form rho
######### makes it into more generalized components "b0xyz"
@nb.jit(nopython=True)
def generalize_rho(rho_array):
    P0, Px, Py, Pz = rho_array
    pxyz = np.array([Px, Py, Pz])
    B0 = 0.5*P0
    bxyz = 0.5*P0*pxyz
    b0xyz = np.array([B0, bxyz[0], bxyz[1], bxyz[2]])
    return b0xyz

######### KEY:
######### this function generalizes then multiplies:
######### first input is rho matrix that is identity minus rho (1-rho)
######### second input is rho matrix that is solo (rho)
######### takes generalized versions and multiplies them in the order (1-rho)rho

def generalize_then_mult(IDrho_array, rho_array):
    a0xyz = generalize_IDrho(IDrho_array)
    b0xyz = generalize_rho(rho_array)
    c0xyz = multiply_matrices(a0xyz, b0xyz)
    C = matrix_maker(c0xyz)
    return C

######### KEY:
######### this function generalizes then multiplies:
######### first input is rho matrix that is identity minus rho (1-rho)
######### second input is rho matrix that is solo (rho)
######### NOTE THE DIFFERENCE:
######### this function takes generalized versions and multiplies them in the order rho(1-rho)

def generalize_then_mult_term2(IDrho_array, rho_array):
    a0xyz = generalize_IDrho(IDrho_array)
    b0xyz = generalize_rho(rho_array)
    c0xyz = multiply_matrices(b0xyz, a0xyz)
    C = matrix_maker(c0xyz)
    return C

In [5]:
#two ways to make final F (term 1) 
#NOTE: these are all not the final Fvvsc, because they have not yet added the harmonic conjugate

##############################################TERM 1

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the first term in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)]
######### it outputs the resulting matrix for this term, but as its ARRAY/VECTOR representation
@nb.jit(nopython=True)
def F_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    a0xyz = generalize_IDrho(p0xyz_2)
    b0xyz = generalize_rho(p0xyz_4)
    c0xyz = multiply_matrices(a0xyz, b0xyz)
    d0xyz = np.array([c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]])
    g0xyz = generalize_IDrho(p0xyz_1)
    h0xyz = generalize_rho(p0xyz_3)
    e0xyz = multiply_matrices(h0xyz, d0xyz)
    f0xyz = multiply_matrices(g0xyz, e0xyz)
    return (f0xyz)

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the first term in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)]
######### it does this by using the components created from "F_components_term1" function then putting it into matrix_maker

def F_array_to_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F = matrix_maker(f0xyz)
    return (F)

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the first term in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)]
######### it outputs the resulting MATRIX for this term

def F_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = generalize_then_mult(p0xyz_2, p0xyz_4)
    D = C + np.trace(C)*identity_matrix
    g0xyz = generalize_IDrho(p0xyz_1)
    h0xyz = generalize_rho(p0xyz_3)
    E = matrix_maker(h0xyz)@D
    F = matrix_maker(g0xyz)@E
    return (F)

##############################################TERM 2

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the second term in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)]
######### it outputs the resulting matrix for this term, but as its ARRAY/VECTOR representation
@nb.jit(nopython=True)
def F_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    a0xyz = generalize_IDrho(p0xyz_4)
    b0xyz = generalize_rho(p0xyz_2)
    c0xyz = multiply_matrices(b0xyz, a0xyz)
    d0xyz = np.array([c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]])
    g0xyz = generalize_IDrho(p0xyz_3)
    h0xyz = generalize_rho(p0xyz_1)
    e0xyz = multiply_matrices(g0xyz, d0xyz)
    f0xyz = multiply_matrices(h0xyz, e0xyz)
    return (f0xyz)

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the second term in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)]
######### it does this by using the components created from "F_components_term2" function then putting it into matrix_maker

def F_array_to_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F = matrix_maker(f0xyz)
    return (F)

                     
######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the second term in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)]

def F_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = generalize_then_mult_term2(p0xyz_4, p0xyz_2)
    D = C + np.trace(C)*identity_matrix
    g0xyz = generalize_IDrho(p0xyz_3)
    h0xyz = generalize_rho(p0xyz_1)
    E = matrix_maker(g0xyz)@D
    F = matrix_maker(h0xyz)@E
    return (F)

In [6]:
# now adding in the harmonic conjugates

##############################################TERM 1

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the FIRST TERM in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)] using F_components_term1
######### then it adds the conjugate of this array, adds it to the original array, and outputs the real array
@nb.jit(nopython=True)
def Fvvsc_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the FIRST TERM in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)] using F_matrix_term1
######### then it adds the harmonic conjugate (conjugate and transpose) of this matrix, adds it to the original matrix, and outputs the hermitian matrix

def Fvvsc_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

######## KEY:
######## takes output array from Fvvsc_components_term1 and makes matrix

def Fvvsc_arr2mat_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

##############################################TERM 2

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the SECOND TERM in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)] using F_components_term2
######### then it adds the harmonic conjugate of this array, adds it to the original array, and outputs the real array
@nb.jit(nopython=True)
def Fvvsc_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the SECOND TERM in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)] using F_matrix_term2
######### then it adds the harmonic conjugate (conjugate and transpose) of this matrix, adds it to the original matrix, and outputs the hermitian matrix

def Fvvsc_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

######## KEY:
######## takes output array from Fvvsc_components_term2 and makes matrix

def Fvvsc_arr2mat_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

In [7]:
#now for the full term!

######## KEY:
######## this creates the full Bennet A.11 equation in ARRAY/VECTOR/COMPONENT form:
######## it does this by taking in those 4 arrays corresponding to 4 matrices
######## and then it uses Fvvsc_components_term1 and Fvvsc_components_term2 
######## to output the full array/vector representatation of the full Fvvsc term
@nb.jit(nopython=True)
def Fvvsc_components(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    term1 = Fvvsc_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    term2 = Fvvsc_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    return term1 + term2

######## KEY:
######## this creates the full Bennet A.11 equation in MATRIX form: 
######## it does this by taking in those 4 arrays corresponding to 4 matrices
######## and then it uses Fvvsc_matrix_term1 and Fvvsc_matrix_term2 
######## to output the full MATRIX of the full Fvvsc term

def Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    term1 = Fvvsc_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    term2 = Fvvsc_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    return term1 + term2

######## KEY:
######## uses Fvvsc_arr2mat_term1 and Fvvsc_arr2mat_term2 to make the full MATRIX of the full Fvvsc term

def Fvvsc_arr2mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    term1 = Fvvsc_arr2mat_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    term2 = Fvvsc_arr2mat_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    return term1 + term2

In [10]:
# now testing
#@nb.jit(nopython=True)
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        random_numbers3 = random_complex_numbers()
        random_numbers4 = random_complex_numbers()

        #compute density matrix both ways   
        L = Fvvsc_arr2mat(random_numbers1, random_numbers2, random_numbers3, random_numbers4)
        M = Fvvsc_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2, random_numbers3, random_numbers4)) #called it something different because i was trying to append random numbers to itslef and it said no
   
    return delta_values, random_numbers_used

In [11]:
N = 100
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [9.412318163124505e-17, 1.7135536217664664e-16, 6.960909316483002e-17, 1.024865329711162e-16, 1.0550193561228064e-16, 8.712474097376328e-17, 3.327824244070837e-16, 7.738863900248638e-17, 7.413054566789849e-17, 8.236742382629704e-17, 8.199700155790194e-17, 1.4309591697273062e-16, 1.4134199028862037e-16, 9.127723653548047e-17, 5.730243635274946e-17, 6.339186890953551e-17, 4.943006430462425e-17, 6.208510334051562e-17, 6.567710449913537e-17, 1.3796409629571208e-16, 8.39502516921455e-17, 5.704003638218276e-17, 8.645868603995307e-17, 8.088423057084089e-17, 1.5626958653186653e-16, 0.0, 1.1459755177522517e-16, 1.915216960318274e-16, 7.120638976412883e-17, 1.5444130560945758e-16, 4.593789129733344e-17, 1.336937620125387e-16, 1.5244149798183414e-16, 8.190495188560874e-17, 2.389703793886986e-16, 7.323878465098425e-17, 7.605167445420492e-17, 1.2352671716682125e-16, 2.6708136200186346e-17, 6.854134618149376e-17, 5.786948445599722e-17, 1.4214022295258867e-16, 1.2422560556526321e-16, 8.